# Script - Migration des données
## Ayoub Errhamani - Rachid Sahli
### BUT SD FA EMS 32 - SAE NoSQL

### Installation et import des librairies

In [ ]:
pip install pymongo pandas 

In [37]:
import sqlite3
import pymongo
import pandas as pd

### Connexion à la base de données SQL

In [38]:
client = pymongo.MongoClient()
db = client.classicmodel
conn = sqlite3.connect("/Users/rs777/Documents/Iut/projets but stid/2024-2025/sae_nosql/ClassicModel.sqlite") # Connexion à la base de données

### Migration de la table *Orders*,*Ordersdetail* et *Products*

In [39]:
# Chargement des données depuis SQLite
orders = pd.read_sql_query("SELECT * FROM Orders;", conn)
order_details = pd.read_sql_query("SELECT * FROM OrderDetails;", conn)
products = pd.read_sql_query("SELECT * FROM Products;", conn)

# Ajout des produits dans les détails des commandes
product = [
    products.query('productCode == @code').to_dict(orient="records") 
    for code in order_details.productCode
]
order_details = order_details.assign(product=product)

# Regroupement des détails des commandes par numéro de commande
orderdetail = [
    order_details.query('orderNumber == @id')
    .drop(columns=["orderNumber"])
    .to_dict(orient="records") 
    for id in orders.orderNumber
]

# Ajout des détails des commandes groupés à la table Orders
orders = orders.assign(orderDetails=orderdetail)

# Insertion des données dans MongoDB
db.order.insert_many(orders.to_dict(orient="records"))

# Vérification des résultats
orders.head()

,orderNumber,orderDate,requiredDate,shippedDate,status,comments,customerNumber,orderDetails
0,10100,2003/1/6 0:00:00,2003/1/13 0:00:00,2003/1/10 0:00:00,Shipped,NULL,363,"[{'productCode': 'S18_1749', 'quantityOrdered'..."
1,10101,2003/1/9 0:00:00,2003/1/18 0:00:00,2003/1/11 0:00:00,Shipped,Check on availability.,128,"[{'productCode': 'S18_2325', 'quantityOrdered'..."
2,10102,2003/1/10 0:00:00,2003/1/18 0:00:00,2003/1/14 0:00:00,Shipped,NULL,181,"[{'productCode': 'S18_1342', 'quantityOrdered'..."
3,10103,2003/1/29 0:00:00,2003/2/7 0:00:00,2003/2/2 0:00:00,Shipped,NULL,121,"[{'productCode': 'S10_1949', 'quantityOrdered'..."
4,10104,2003/1/31 0:00:00,2003/2/9 0:00:00,2003/2/1 0:00:00,Shipped,NULL,141,"[{'productCode': 'S12_3148', 'quantityOrdered'..."


### Migration des tables Employes et Offices

In [40]:
# Chargement des données depuis SQLite
employees = pd.read_sql_query("SELECT * FROM Employees;", conn)
offices = pd.read_sql_query("SELECT * FROM Offices;", conn)

office = [
    offices.query('officeCode == @id')
        .drop(columns = ["officeCode"])
        .to_dict(orient = "records") 
        for id in employees.officeCode
]

# Vérification des résultats
print(office)

[[{'city': 'San Francisco', 'phone': '+1 650 219 4782', 'addressLine1': '100 Market Street', 'addressLine2': 'Suite 300', 'state': 'CA', 'country': 'USA', 'postalCode': '94080', 'territory': None}], [{'city': 'San Francisco', 'phone': '+1 650 219 4782', 'addressLine1': '100 Market Street', 'addressLine2': 'Suite 300', 'state': 'CA', 'country': 'USA', 'postalCode': '94080', 'territory': None}], [{'city': 'San Francisco', 'phone': '+1 650 219 4782', 'addressLine1': '100 Market Street', 'addressLine2': 'Suite 300', 'state': 'CA', 'country': 'USA', 'postalCode': '94080', 'territory': None}], [{'city': 'Sydney', 'phone': '+61 2 9264 2451', 'addressLine1': '5-11 Wentworth Avenue', 'addressLine2': 'Floor #2', 'state': 'NULL', 'country': 'Australia', 'postalCode': 'NSW 2010', 'territory': 'APAC'}], [{'city': 'Paris', 'phone': '+33 14 723 4404', 'addressLine1': "43 Rue Jouffroy D'abbans", 'addressLine2': 'NULL', 'state': None, 'country': 'France', 'postalCode': '75017', 'territory': 'EMEA'}], [

In [41]:
# Ajout de la colonne office
employee = employees.assign(offices= office)

# Vérification des résultats
employee.head()

,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle,offices
0,1002,Murphy,Diane,x5800,dmurphy@classicmodelcars.com,1.0,NULL,President,"[{'city': 'San Francisco', 'phone': '+1 650 21..."
1,1056,Patterson,Mary,x4611,mpatterso@classicmodelcars.com,1.0,1002,VP Sales,"[{'city': 'San Francisco', 'phone': '+1 650 21..."
2,1076,Firrelli,Jeff,x9273,jfirrelli@classicmodelcars.com,1.0,1002,VP Marketing,"[{'city': 'San Francisco', 'phone': '+1 650 21..."
3,1088,Patterson,William,x4871,wpatterson@classicmodelcars.com,6.0,1056,"Sales Manager (JAPAN, APAC)","[{'city': 'Sydney', 'phone': '+61 2 9264 2451'..."
4,1102,Bondur,Gerard,x5408,athompson@classicmodelcars.com,4.0,1056,Sale Manager (EMEA),"[{'city': 'Paris', 'phone': '+33 14 723 4404',..."


In [42]:
# Insertion de la table employee dans mongoDB
db.employee.insert_many(
    employee.to_dict(orient = "records")
)

InsertManyResult([ObjectId('67672ffd5283ed5716b47348'), ObjectId('67672ffd5283ed5716b47349'), ObjectId('67672ffd5283ed5716b4734a'), ObjectId('67672ffd5283ed5716b4734b'), ObjectId('67672ffd5283ed5716b4734c'), ObjectId('67672ffd5283ed5716b4734d'), ObjectId('67672ffd5283ed5716b4734e'), ObjectId('67672ffd5283ed5716b4734f'), ObjectId('67672ffd5283ed5716b47350'), ObjectId('67672ffd5283ed5716b47351'), ObjectId('67672ffd5283ed5716b47352'), ObjectId('67672ffd5283ed5716b47353'), ObjectId('67672ffd5283ed5716b47354'), ObjectId('67672ffd5283ed5716b47355'), ObjectId('67672ffd5283ed5716b47356'), ObjectId('67672ffd5283ed5716b47357'), ObjectId('67672ffd5283ed5716b47358'), ObjectId('67672ffd5283ed5716b47359'), ObjectId('67672ffd5283ed5716b4735a'), ObjectId('67672ffd5283ed5716b4735b'), ObjectId('67672ffd5283ed5716b4735c'), ObjectId('67672ffd5283ed5716b4735d'), ObjectId('67672ffd5283ed5716b4735e')], acknowledged=True)

### Migration de la table Payments

In [43]:
# Chargement des données depuis SQLite
payments = pd.read_sql_query("SELECT * FROM Payments;", conn)

# Convertion en dictionnaires
payments.to_dict(orient = "records") 

# Vérification des résultats
payments.head()

,customerNumber,checkNumber,paymentDate,amount
0,103,HQ336336,2004/10/19 0:00:00,5307.98
1,103,JM555205,2003/6/5 0:00:00,16560.30
2,103,OM314933,2004/12/18 0:00:00,2311.68
3,112,BO864823,2004/12/17 0:00:00,14449.61
4,112,HQ55022,2003/6/6 0:00:00,33847.62


In [44]:
# Insertion de la table payments dans mongoDB
db.payments.insert_many(payments.to_dict(orient = "records") )

InsertManyResult([ObjectId('676730055283ed5716b4735f'), ObjectId('676730055283ed5716b47360'), ObjectId('676730055283ed5716b47361'), ObjectId('676730055283ed5716b47362'), ObjectId('676730055283ed5716b47363'), ObjectId('676730055283ed5716b47364'), ObjectId('676730055283ed5716b47365'), ObjectId('676730055283ed5716b47366'), ObjectId('676730055283ed5716b47367'), ObjectId('676730055283ed5716b47368'), ObjectId('676730055283ed5716b47369'), ObjectId('676730055283ed5716b4736a'), ObjectId('676730055283ed5716b4736b'), ObjectId('676730055283ed5716b4736c'), ObjectId('676730055283ed5716b4736d'), ObjectId('676730055283ed5716b4736e'), ObjectId('676730055283ed5716b4736f'), ObjectId('676730055283ed5716b47370'), ObjectId('676730055283ed5716b47371'), ObjectId('676730055283ed5716b47372'), ObjectId('676730055283ed5716b47373'), ObjectId('676730055283ed5716b47374'), ObjectId('676730055283ed5716b47375'), ObjectId('676730055283ed5716b47376'), ObjectId('676730055283ed5716b47377'), ObjectId('676730055283ed5716b473

### Migration de la table Customers

In [46]:
# Chargement des données depuis SQLite
customers = pd.read_sql_query("SELECT * FROM Customers;", conn)

# Convertion en dictionnaires
customers.to_dict(orient = "records") 

# Vérification des résultats
customers.head()

InsertManyResult([ObjectId('676730645283ed5716b4746f'), ObjectId('676730645283ed5716b47470'), ObjectId('676730645283ed5716b47471'), ObjectId('676730645283ed5716b47472'), ObjectId('676730645283ed5716b47473'), ObjectId('676730645283ed5716b47474'), ObjectId('676730645283ed5716b47475'), ObjectId('676730645283ed5716b47476'), ObjectId('676730645283ed5716b47477'), ObjectId('676730645283ed5716b47478'), ObjectId('676730645283ed5716b47479'), ObjectId('676730645283ed5716b4747a'), ObjectId('676730645283ed5716b4747b'), ObjectId('676730645283ed5716b4747c'), ObjectId('676730645283ed5716b4747d'), ObjectId('676730645283ed5716b4747e'), ObjectId('676730645283ed5716b4747f'), ObjectId('676730645283ed5716b47480'), ObjectId('676730645283ed5716b47481'), ObjectId('676730645283ed5716b47482'), ObjectId('676730645283ed5716b47483'), ObjectId('676730645283ed5716b47484'), ObjectId('676730645283ed5716b47485'), ObjectId('676730645283ed5716b47486'), ObjectId('676730645283ed5716b47487'), ObjectId('676730645283ed5716b474

In [ ]:
# Insertion de la table payments dans mongoDB
db.customers.insert_many(customers.to_dict(orient = "records") )